# Google Earth Engine Variable Extraction Notebook

This notebook provides functions to extract various environmental variables from Google Earth Engine (GEE) datasets.
The primary goals are:
- Define functions to retrieve specific variables based on user-defined parameters (region, time period, frequency).
- For time-series data, aggregate it to hourly, daily, monthly, or yearly means.
- Save the extracted data for each variable into a separate CSV file.

**Instructions:**
1. Run the GEE Authentication and Initialization cell first. You will need to authenticate with a Google account that has GEE access.
2. Define your region of interest (AOI) as a GeoJSON-like dictionary.
3. Call the specific extraction functions for the variables you need, providing the AOI, date range, frequency, and output directory.

In [38]:
# Run this once per session
# Update folium to latest version

!sudo apt-get install wkhtmltopdf
!pip install folium html2image
!pip install --upgrade folium
!pip install playwright
!playwright install

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wkhtmltopdf is already the newest version (0.12.6-2).
0 upgraded, 0 newly installed, 0 to remove and 90 not upgraded.


In [27]:
import ee
import pandas as pd
import numpy as np
import datetime
from dateutil.relativedelta import relativedelta
import os
import csv
import math # Ensure math is imported for potential calculations like wind
import calendar

In [28]:
# Trigger the authentication flow.
# This will print a URL, open it, authorize, and copy the code back into the input box.
try:
    ee.Authenticate()
except Exception as e:
    print(f"Authentication failed or already authenticated: {e}")
    # For automated environments or if auth was done in a previous session,
    # this might raise an error if not needed, so we can often proceed.

# Initialize the library.
# Replace 'YOUR_GEE_PROJECT' with your actual GEE project ID if you have one,
# otherwise, GEE often can use a default cloud project associated with your account.
try:
    ee.Initialize(project='gen-lang-client-0253961861')
    print("GEE Initialized successfully.")
except Exception as e:
    try:
        # Fallback if project-specific initialization fails
        ee.Initialize()
        print("GEE Initialized successfully (default project).")
    except Exception as e_init:
        print(f"GEE Initialization failed: {e_init}")
        print("Please ensure you have authenticated and have a GEE-enabled project.")

# Define a helper to check GEE initialization status
def check_gee_initialized():
    try:
        ee.ImageCollection('MODIS/061/MCD43A3').limit(1).size().getInfo()
        print("GEE is initialized and accessible.")
        return True
    except Exception as e:
        print(f"GEE not properly initialized or accessible: {e}")
        return False

check_gee_initialized()

GEE Initialized successfully.
GEE is initialized and accessible.


True

In [29]:
# Generic Time Series Extraction Function
def extract_gee_time_series(
    variable_name: str,
    region_geojson: dict, # GeoJSON dictionary for the region
    start_date_str: str,
    end_date_str: str,
    frequency: str,  # 'hourly', 'daily', 'monthly', 'yearly'
    gee_dataset_id: str,
    gee_band_name: str, # Can be a list for multi-band calculations (e.g. wind)
    scale: int,
    output_dir: str,
    gee_project: str = None, # Optional: GEE project for initialization if needed
    reducer: ee.Reducer = ee.Reducer.mean(),
    data_scaling_factor: float = None,
    data_offset_factor: float = None,
    post_process_function: callable = None, # Function to apply to the reduced value or dictionary of values
    nan_value = None # Value to use if GEE returns None
) -> str:
    """
    Extracts time-series data from Google Earth Engine for a specified variable and frequency,
    saves it to a CSV file, and returns the path to the CSV.

    Parameters:
    - variable_name: Name of the variable (used for CSV filename).
    - region_geojson: GeoJSON dictionary defining the region of interest.
    - start_date_str: Start date in 'YYYY-MM-DD' format.
    - end_date_str: End date in 'YYYY-MM-DD' format.
    - frequency: Aggregation frequency ('hourly', 'daily', 'monthly', 'yearly').
    - gee_dataset_id: Earth Engine ImageCollection ID.
    - gee_band_name: Name of the band(s) to extract. If a list, post_process_function must handle it.
    - scale: Spatial resolution in meters for reduction.
    - output_dir: Directory to save the output CSV file.
    - gee_project: Optional GEE project ID for ee.Initialize().
    - reducer: Earth Engine reducer to apply (default: ee.Reducer.mean()).
    - data_scaling_factor: Optional factor to multiply the band data by.
    - data_offset_factor: Optional offset to add to the band data.
    - post_process_function: Optional function to apply to the raw reduced value(s).
                             It should accept a dictionary of band values if multiple bands are processed,
                             or a single value if one band is processed. It should return a dictionary
                             of processed values or a single processed value.
    - nan_value: Value to fill in if GEE returns no data for a period.

    Returns:
    - Path to the generated CSV file.
    """
    if gee_project:
        try:
            ee.Initialize(project=gee_project)
        except Exception:
            ee.Initialize() # Fallback

    ee_region = ee.Geometry(region_geojson)

    start_date = datetime.datetime.strptime(start_date_str, '%Y-%m-%d')
    end_date = datetime.datetime.strptime(end_date_str, '%Y-%m-%d')

    os.makedirs(output_dir, exist_ok=True)

    date_periods = []
    current_date = start_date

    if frequency == 'hourly':
        delta = relativedelta(hours=1)
        date_format_label = '%Y-%m-%d %H:%M:%S'
        while current_date <= end_date:
            period_end_date = current_date + delta - relativedelta(seconds=1) # End of the hour
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "end": period_end_date.strftime('%Y-%m-%dT%H:%M:%S'),
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'daily':
        delta = relativedelta(days=1)
        date_format_label = '%Y-%m-%d'
        while current_date <= end_date:
            date_periods.append({
                "start": current_date.strftime('%Y-%m-%d'),
                "end": (current_date + delta - relativedelta(days=1)).strftime('%Y-%m-%d'), # inclusive end
                "label": current_date.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'monthly':
        delta = relativedelta(months=1)
        date_format_label = '%Y-%m'
        while current_date <= end_date:
            month_start = current_date.replace(day=1)
            month_end = month_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": month_start.strftime('%Y-%m-%d'),
                "end": month_end.strftime('%Y-%m-%d'),
                "label": month_start.strftime(date_format_label)
            })
            current_date += delta
    elif frequency == 'yearly':
        delta = relativedelta(years=1)
        date_format_label = '%Y'
        while current_date <= end_date:
            year_start = current_date.replace(month=1, day=1)
            year_end = year_start + delta - relativedelta(days=1)
            date_periods.append({
                "start": year_start.strftime('%Y-%m-%d'),
                "end": year_end.strftime('%Y-%m-%d'),
                "label": year_start.strftime(date_format_label)
            })
            current_date += delta
    else:
        raise ValueError("Invalid frequency. Choose from 'hourly', 'daily', 'monthly', 'yearly'.")

    results = []

    for period in date_periods:
        try:
            collection = ee.ImageCollection(gee_dataset_id) \
                           .filterDate(ee.Date(period["start"]), ee.Date(period["end"]).advance(1, 'day')) # GEE end date is exclusive

            if isinstance(gee_band_name, list): # For multi-band variables like wind
                selected_bands_collection = collection.select(gee_band_name)
            else: # Single band
                selected_bands_collection = collection.select([gee_band_name])

            # Check if collection is empty for the period
            if selected_bands_collection.size().getInfo() == 0:
                print(f"No images found for {variable_name} in period {period['label']} for dataset {gee_dataset_id}")
                reduced_value = nan_value
                if isinstance(gee_band_name, list) and nan_value is not None:
                     # If multiple bands expected, fill with nan_value for each
                    reduced_value = {band: nan_value for band in gee_band_name}

                if post_process_function and reduced_value is not None : # nan_value can be processed if needed
                     processed_value = post_process_function(reduced_value)
                else:
                     processed_value = reduced_value

                # Ensure processed_value is a dictionary for DataFrame creation
                if not isinstance(processed_value, dict) and isinstance(gee_band_name, list):
                    # if single value came from post_process for multiple bands, try to map it
                    # this might need adjustment based on post_process_function's behavior
                    processed_value = {f"{variable_name}_{b}" if len(gee_band_name) > 1 else variable_name : processed_value for b in gee_band_name}
                elif not isinstance(processed_value, dict):
                    processed_value = {variable_name: processed_value}

            else:
                image_for_period = selected_bands_collection.mean() # Temporal aggregation

                if data_scaling_factor is not None:
                    image_for_period = image_for_period.multiply(data_scaling_factor)
                if data_offset_factor is not None:
                    image_for_period = image_for_period.add(data_offset_factor)

                # Perform reduction
                reduction = image_for_period.reduceRegion(
                    reducer=reducer,
                    geometry=ee_region,
                    scale=scale,
                    maxPixels=1e10,
                    bestEffort=True, # Added bestEffort
                    tileScale=0.1 # Added tileScale
                )

                raw_reduced_value = {}
                if isinstance(gee_band_name, list):
                    for band in gee_band_name:
                        raw_reduced_value[band] = reduction.get(band).getInfo()
                else:
                    raw_reduced_value = reduction.get(gee_band_name).getInfo()

                if post_process_function:
                    processed_value = post_process_function(raw_reduced_value)
                else:
                    processed_value = raw_reduced_value

            # Structure for DataFrame
            current_row = {'timestamp': period['label']}
            if isinstance(processed_value, dict):
                current_row.update(processed_value)
            else: # Single value result
                current_row[variable_name] = processed_value
            results.append(current_row)

        except Exception as e:
            print(f"Error processing period {period['label']} for {variable_name}: {e}")
            # Add a row with nan_value or error indication
            error_entry = {'timestamp': period['label']}
            if isinstance(gee_band_name, list):
                for band in gee_band_name:
                    error_entry[f"{variable_name}_{band}"] = nan_value  # Or some error string
            else:
                error_entry[variable_name] = nan_value # Or some error string
            results.append(error_entry)
            continue # Continue to next period

    if not results:
        print(f"No data extracted for {variable_name} in the given period.")
        return None

    df = pd.DataFrame(results)
    # Ensure timestamp is the first column
    cols = ['timestamp'] + [col for col in df.columns if col != 'timestamp']
    df = df[cols]

    # Sanitize filename
    safe_start_date = start_date_str.replace('-', '')
    safe_end_date = end_date_str.replace('-', '')
    csv_filename = f"{variable_name}_{frequency}_{safe_start_date}_{safe_end_date}.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    df.to_csv(csv_path, index=False, na_rep=str(nan_value) if nan_value is not None else 'NaN') # Use nan_value for na_rep

    print(f"Successfully saved data for {variable_name} to {csv_path}")
    return csv_path

# --- Helper function for MODIS LST to Celsius ---
def convert_modis_lst_to_celsius(lst_value):
    """Converts MODIS LST (scaled Kelvin) to Celsius."""
    if lst_value is None:
        return None
    return (lst_value * 0.02) - 273.15

# --- Specific Variable Extraction Functions ---

# Albedo
def extract_albedo_bsa(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=500):
    return extract_gee_time_series(
        variable_name="Albedo_BSA",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MCD43A3', # As per reference notebook
        gee_band_name='Albedo_BSA_shortwave', # As per reference notebook
        scale=scale,
        output_dir=output_dir,
        data_scaling_factor=0.001, # MODIS albedo scaling
        nan_value=np.nan # Use numpy's NaN for missing values
    )

def extract_albedo_wsa(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=500):
    return extract_gee_time_series(
        variable_name="Albedo_WSA",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MCD43A3', # As per reference notebook
        gee_band_name='Albedo_WSA_shortwave', # As per reference notebook
        scale=scale,
        output_dir=output_dir,
        data_scaling_factor=0.001, # MODIS albedo scaling
        nan_value=np.nan
    )

# Solar Radiation (using LST Day as proxy, similar to reference notebook)
# Note: This is an approximation. A direct solar radiation dataset would be better if available and suitable.
def post_process_solar_radiation(lst_day_value):
    """Approximates solar radiation from MODIS LST Day value."""
    if lst_day_value is None:
        return None
    # If input is a dict (from multi-band processing in generic func),
    # extract the value. This handles cases where post_process_function
    # might receive a dict even for single-band selection if the generic
    # function's internal logic changes or if it's called directly with a dict.
    if isinstance(lst_day_value, dict):
        val = lst_day_value.get('LST_Day_1km', None)
        if val is None: return None
    else:
        val = lst_day_value

    temp_kelvin = val * 0.02 # MODIS LST scaling to Kelvin
    stefan_boltzmann = 5.67e-8  # W/(m^2 K^4)
    # This is a simplified Stefan-Boltzmann law application, assuming emissivity = 1
    # The reference notebook had this calculation. True GHI/DNI would come from datasets like ERA5 or specific solar datasets.
    return stefan_boltzmann * (temp_kelvin ** 4)

def extract_radiacion_solar(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Radiacion_Solar_Approximated_from_LST",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # Using LST dataset as per reference
        gee_band_name='LST_Day_1km',       # Using LST Day band
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_solar_radiation,
        nan_value=np.nan
    )

# Temperature
def extract_temperatura_dia(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Temperatura_Dia_Celsius",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # As per reference notebook
        gee_band_name='LST_Day_1km',
        scale=scale,
        output_dir=output_dir,
        post_process_function=lambda x: convert_modis_lst_to_celsius(x.get('LST_Day_1km') if isinstance(x, dict) else x) if x is not None else None, # LST to Celsius
        nan_value=np.nan
    )

def extract_temperatura_noche(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=1000):
    return extract_gee_time_series(
        variable_name="Temperatura_Noche_Celsius",
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='MODIS/061/MOD11A1', # As per reference notebook
        gee_band_name='LST_Night_1km',
        scale=scale,
        output_dir=output_dir,
        post_process_function=lambda x: convert_modis_lst_to_celsius(x.get('LST_Night_1km') if isinstance(x, dict) else x) if x is not None else None, # LST to Celsius
        nan_value=np.nan
    )

In [30]:
# --- Wind Speed and Direction ---
def post_process_wind_data(wind_components):
    """Calculates wind speed and direction from u and v components."""
    u = wind_components.get('u_component_of_wind_10m')
    v = wind_components.get('v_component_of_wind_10m')

    if u is None or v is None:
        return {'Viento_Velocidad': np.nan, 'Viento_Direccion': np.nan}

    speed = math.sqrt(u**2 + v**2)
    # Wind direction: meteorological convention (degrees from North, clockwise)
    # atan2(u,v) gives angle w.r.t positive y-axis (North). Convert to degrees.
    # Then adjust to be 0-360.
    # direction_rad = math.atan2(u, v) # u is eastward, v is northward
    # direction_deg = math.degrees(direction_rad)
    # direction_met = (270 - direction_deg) % 360 # As in reference notebook
    # A common formula for meteorological wind direction from u,v components:
    direction_met = (180 / math.pi) * math.atan2(-u, -v) + 180
    direction_met = direction_met % 360 # Ensure it's within 0-360

    return {'Viento_Velocidad': speed, 'Viento_Direccion': direction_met}

def extract_viento(region_geojson, start_date_str, end_date_str, frequency, output_dir, scale=10000):
    # Note: ERA5 Land is hourly. If 'daily', 'monthly', 'yearly' frequency is requested,
    # the generic function will average the hourly u/v components first, then calculate speed/direction.
    return extract_gee_time_series(
        variable_name="Viento", # Base name, will be expanded by post_process_wind_data keys
        region_geojson=region_geojson,
        start_date_str=start_date_str,
        end_date_str=end_date_str,
        frequency=frequency,
        gee_dataset_id='ECMWF/ERA5_LAND/HOURLY', # As per reference notebook
        gee_band_name=['u_component_of_wind_10m', 'v_component_of_wind_10m'],
        scale=scale,
        output_dir=output_dir,
        post_process_function=post_process_wind_data,
        nan_value=np.nan
    )

# --- Topography (Elevation, Slope, Aspect) ---
# These are static, so they don't depend on date range or frequency in the same way.
# The function will calculate mean values over the region for a single point in time (the SRTM image is static).
def extract_topography(region_geojson, output_dir, scale=30, variable_name_prefix="Topografia_"):
    """
    Extracts mean Elevation, Slope, and Aspect for a region and saves to a CSV.
    Output CSV will have one row with columns: 'Elevation', 'Slope', 'Aspect'.
    """
    ee_region = ee.Geometry(region_geojson)
    os.makedirs(output_dir, exist_ok=True)

    try:
        srtm = ee.Image('USGS/SRTMGL1_003') # SRTM is a single image
        elevation = srtm.select('elevation')
        slope = ee.Terrain.slope(elevation)
        aspect = ee.Terrain.aspect(elevation)

        topography_image = ee.Image.cat([elevation, slope, aspect]).rename(['Elevacion', 'Pendiente', 'Aspecto'])

        reduction = topography_image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=ee_region,
            scale=scale,
            maxPixels=1e10,
            bestEffort=True,
            tileScale=0.1
        )

        # GetInfo once
        reduced_data = reduction.getInfo()

        # Handle cases where a key might be missing (though unlikely for these specific bands from SRTM)
        data_for_df = {
            'Elevacion': reduced_data.get('Elevacion', np.nan),
            'Pendiente': reduced_data.get('Pendiente', np.nan),
            'Aspecto': reduced_data.get('Aspecto', np.nan)
        }

        df = pd.DataFrame([data_for_df])

        # Define a simple, non-temporal filename
        csv_filename = f"{variable_name_prefix}mean_values.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))

        print(f"Successfully saved topography data to {csv_path}")
        return csv_path

    except Exception as e:
        print(f"Error extracting topography data: {e}")
        # Create a CSV with NaNs if there's an error
        df = pd.DataFrame([{'Elevacion': np.nan, 'Pendiente': np.nan, 'Aspecto': np.nan}])
        csv_filename = f"{variable_name_prefix}mean_values_error.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))
        return csv_path


# --- Land Cover (Principal Type and Percentage) ---
# This is typically analyzed for a specific year or period, not as a continuous time series like temperature.
# The function will find the dominant land cover type and its percentage for a given year.
def extract_land_cover(region_geojson, year_str, output_dir, scale=500, variable_name_prefix="Cobertura_"):
    """
    Extracts the principal land cover type and its percentage for a region and year.
    Saves to a CSV with columns: 'Year', 'Cobertura_Terrestre_Principal', 'Cobertura_Terrestre_Porcentaje'.
    Uses MODIS MCD12Q1 dataset.
    """
    ee_region = ee.Geometry(region_geojson)
    os.makedirs(output_dir, exist_ok=True)

    try:
        # MODIS Land Cover Type 1 (IGBP classification)
        # MCD12Q1 provides yearly data. Filter for the specific year.
        start_date = f"{year_str}-01-01"
        end_date = f"{year_str}-12-31" # End of the year

        land_cover_collection = ee.ImageCollection('MODIS/061/MCD12Q1') \
                                  .filterDate(start_date, end_date) \
                                  .select('LC_Type1') # IGBP classification band

        # Get the image for the year (should be one)
        land_cover_image = land_cover_collection.first()

        # A robust check to see if the image is valid and has bands
        # Attempt to get band names; if it fails or is empty, image is likely invalid/empty
        valid_image = False
        try:
            if land_cover_image.bandNames().size().getInfo() > 0:
                valid_image = True
        except Exception: # Handles cases where land_cover_image might be null or not a proper image
            valid_image = False

        if not valid_image:
             print(f"No MODIS land cover data found or image is invalid for year {year_str}.")
             data_for_df = {'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}
        else:
            # Calculate frequency histogram of land cover types in the region
            histogram = land_cover_image.reduceRegion(
                reducer=ee.Reducer.frequencyHistogram(),
                geometry=ee_region,
                scale=scale,
                maxPixels=1e10,
                bestEffort=True,
                tileScale=0.1
            ).get('LC_Type1') # Get the histogram for the band

            histogram_info = histogram.getInfo() # This can be slow for very large regions

            if not histogram_info: # Check if histogram is empty
                print(f"Land cover histogram is empty for year {year_str} in the region.")
                data_for_df = {'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}
            else:
                # Convert keys (class IDs) to integers and find the principal class
                class_counts = {int(k): v for k, v in histogram_info.items()}
                total_pixels = sum(class_counts.values())

                if total_pixels == 0:
                    principal_class_id = np.nan
                    percentage = np.nan
                else:
                    principal_class_id = max(class_counts, key=class_counts.get)
                    percentage = (class_counts[principal_class_id] / total_pixels) * 100

                data_for_df = {
                    'Year': year_str,
                    'Cobertura_Terrestre_Principal': principal_class_id,
                    'Cobertura_Terrestre_Porcentaje': percentage
                }

        df = pd.DataFrame([data_for_df])
        csv_filename = f"{variable_name_prefix}{year_str}.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))

        print(f"Successfully saved land cover data for {year_str} to {csv_path}")
        return csv_path

    except Exception as e:
        print(f"Error extracting land cover data for {year_str}: {e}")
        df = pd.DataFrame([{'Year': year_str, 'Cobertura_Terrestre_Principal': np.nan, 'Cobertura_Terrestre_Porcentaje': np.nan}])
        csv_filename = f"{variable_name_prefix}{year_str}_error.csv"
        csv_path = os.path.join(output_dir, csv_filename)
        df.to_csv(csv_path, index=False, na_rep=str(np.nan))
        return csv_path


---
## Example Usage of Extraction Functions

Below are examples of how to use the implemented functions.
- You'll need to define your `region_of_interest` (as a GeoJSON-like Python dictionary).
- Specify your desired `start_date`, `end_date`, `year_for_landcover`, and `output_directory`.
- Uncomment the function calls you wish to run.
- Ensure the `output_directory` exists or the functions will create it.

In [31]:
import math
import re

def dms_to_decimal(coord_str):
    # First, try to parse as a decimal degree with optional direction
    decimal_pattern = re.compile(
        r'^\s*([+-]?[\d.]+)\s*([NSEW]?)\s*$',
        re.IGNORECASE
    )
    decimal_match = decimal_pattern.match(coord_str.strip())
    if decimal_match:
        number_str, direction = decimal_match.groups()
        try:
            number = float(number_str)
        except ValueError:
            pass  # Not a valid decimal, proceed to check DMS
        else:
            if direction:
                direction = direction.upper()
                decimal = abs(number)
                if direction in ['S', 'W']:
                    decimal *= -1
                return decimal
            else:
                return number

    # If not a decimal, attempt to parse as DMS
    dms_pattern = re.compile(
        r'''\s*([+-]?\d+)\s*°\s*([+-]?\d+)\s*'\s*([+-]?\d+\.?\d*)\s*"*\s*([NSEW]?)\s*''',
        re.IGNORECASE
    )
    dms_match = dms_pattern.match(coord_str.strip())
    if not dms_match:
        raise ValueError(f"Invalid coordinate format: {coord_str}")

    degrees, minutes, seconds, direction = dms_match.groups()
    degrees = float(degrees)
    minutes = float(minutes)
    seconds = float(seconds)
    decimal = degrees + minutes / 60 + seconds / 3600

    if direction.upper() in ['S', 'W']:
        decimal *= -1

    return decimal

def get_point(center_lat, center_lon, distance_km, bearing_deg):
    R = 6371  # Earth radius in kilometers
    delta = distance_km / R  # Angular distance in radians

    lat1 = math.radians(center_lat)
    lon1 = math.radians(center_lon)
    theta = math.radians(bearing_deg)

    lat2 = math.asin(
        math.sin(lat1) * math.cos(delta) +
        math.cos(lat1) * math.sin(delta) * math.cos(theta))
    lon2 = lon1 + math.atan2(
        math.sin(theta) * math.sin(delta) * math.cos(lat1),
        math.cos(delta) - math.sin(lat1) * math.sin(lat2))

    lon2 = (lon2 + 3 * math.pi) % (2 * math.pi) - math.pi

    return (math.degrees(lat2), math.degrees(lon2))

def generate_polygon_vertices(lat_input, lon_input, num_points, distance_km):
    # Convert inputs to decimal degrees
    try:
        center_lat = dms_to_decimal(lat_input)
        center_lon = dms_to_decimal(lon_input)
    except ValueError as e:
        raise ValueError(f"Invalid coordinate format: {e}")

    vertices = []
    for i in range(num_points):
        bearing = (i * 360.0) / num_points
        lat, lon = get_point(center_lat, center_lon, distance_km, bearing)
        vertices.append((lat, lon))
    return vertices

In [32]:
generate_polygon_vertices('16.697997', '42.794972', 4, 5)

[(16.742963080295937, 42.79497199999995),
 (16.697991706970537, 42.841917658644434),
 (16.65303091970406, 42.79497199999995),
 (16.697991706970537, 42.74802634135547)]

In [33]:
generate_polygon_vertices("16° 41' 53""", "42° 47' 42""", 4, 5)

[(16.74302163585149, 42.79499999999996),
 (16.698050262506435, 42.84194567303673),
 (16.653089475259613, 42.79499999999996),
 (16.698050262506435, 42.748054326963285)]

In [34]:
def convert_vertices_to_geojson(vertices):
    """
    Convert polygon vertices to a GeoJSON-like Polygon dictionary.

    Args:
        vertices (list): List of (latitude, longitude) tuples

    Returns:
        dict: GeoJSON-like Polygon dictionary
    """
    # Convert (latitude, longitude) to (longitude, latitude) for GeoJSON
    coordinates = [[lon, lat] for lat, lon in vertices]

    # Ensure the polygon is closed by appending the first coordinate if necessary
    if coordinates:
        if coordinates[0] != coordinates[-1]:
            coordinates.append(coordinates[0])

    return {
        'type': 'Polygon',
        'coordinates': [coordinates]
    }

In [35]:
vertices = generate_polygon_vertices('16.697997', '42.794972', 4, 5)
poligon_AOI = convert_vertices_to_geojson(vertices)
poligon_AOI

{'type': 'Polygon',
 'coordinates': [[[42.79497199999995, 16.742963080295937],
   [42.841917658644434, 16.697991706970537],
   [42.79497199999995, 16.65303091970406],
   [42.74802634135547, 16.697991706970537],
   [42.79497199999995, 16.742963080295937]]]}

In [40]:
import folium
from folium import Figure
from html2image import Html2Image

def visualize_with_folium(vertices, center_lat=None, center_lon=None,
                          zoom_start=12, save_path="map.html"):
    """
    Creates a Folium map with a polygon and saves it as an HTML file.

    :param vertices: List of [lat, lon] pairs defining the polygon
    :param center_lat: Center latitude (optional)
    :param center_lon: Center longitude (optional)
    :param zoom_start: Initial zoom level
    :param save_path: Path to save the HTML file
    :return: Folium Map object
    """
    # Calculate center if not provided
    if center_lat is None or center_lon is None:
        center_lat = sum(v[0] for v in vertices) / len(vertices)
        center_lon = sum(v[1] for v in vertices) / len(vertices)

    # Create a Figure container
    figure = Figure(width=800, height=600)

    # Create base map inside the figure
    map_obj = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=zoom_start,
        tiles='Esri.WorldImagery'
    )
    figure.add_child(map_obj)

    # Add elements
    folium.Marker(
        [center_lat, center_lon],
        popup="Center",
        icon=folium.Icon(color='red')
    ).add_to(map_obj)

    folium.Polygon(
        vertices,
        color='#ff0000',
        fill=True,
        fill_opacity=0.2
    ).add_to(map_obj)

    # Save HTML
    map_obj.save(save_path)

    return map_obj

In [50]:
# Generate vertices
vertices = generate_polygon_vertices(
    lat_input="24° 31' 33\" N",
    lon_input="40° 44' 48\" E",
    num_points=4,
    distance_km=5
)

# Define paths
png_output_dir = os.getcwd()
png_file_path = os.path.join(png_output_dir, "map.png")
html_map_path = os.path.join(png_output_dir,"map.html")


# Create visualization and save HTML
map_obj = visualize_with_folium(
    vertices=vertices,
    save_path=html_map_path,
    zoom_start=12
)

# Ensure the output directory exists
os.makedirs(png_output_dir, exist_ok=True)

# Initialize Html2Image
# The previous initialization hti = Html2Image() is sufficient if wkhtmltoimage is in PATH.
# The 'wkhtmltoimage' keyword argument is not accepted by the constructor.
# If wkhtmltoimage is not found, consider adding it to your system's PATH or
# setting the WKHTMLTOIMAGE_PATH environment variable.
hti = Html2Image(output_path=png_output_dir)

# Generate the PNG from the saved HTML using Html2Image
hti.screenshot(
    html_file=html_map_path,
    save_as='map.png',
    size=(800, 600)
)

print(f"PNG file has been saved to: {png_file_path}")

PNG file has been saved to: /content/map.png


In [58]:
import asyncio
from playwright.async_api import async_playwright
import os
import nest_asyncio # Import nest_asyncio

# Apply nest_asyncio to allow asyncio to run inside a loop (like Jupyter)
nest_asyncio.apply()

async def take_screenshot(html_path, output_png_path, width=800, height=600):
    async with async_playwright() as p:
        browser = await p.chromium.launch() # Or firefox, webkit

        # --- Add this line to create a BrowserContext ---
        context = await browser.new_context(viewport={'width': width, 'height': height})
        # --- End of added line ---

        # --- Modify this line to create the page from the context ---
        page = await context.new_page() # Create the page within the context
        # --- End of modified line ---

        # Construct file URL
        file_url = f'file://{os.path.abspath(html_path)}'
        print(f"Attempting to load HTML file from: {file_url}") # Debugging print

        try:
            await page.goto(file_url)
            print(f"Successfully loaded {html_path}")
            await page.screenshot(path=output_png_path)
            print(f"Screenshot saved to: {output_png_path}")
        except Exception as e:
            print(f"Error during screenshot: {e}")
        finally:
            # --- Close the context and then the browser ---
            await context.close() # Close the context
            await browser.close() # Close the browser
            # --- End of closing lines ---
            print("Browser and context closed.")


In [59]:
# Now run the async function
# You can use asyncio.run() after applying nest_asyncio, or the %autoawait magic

# Using asyncio.run()
try:
    asyncio.run(take_screenshot(html_map_path, png_file_path, width=800, height=600))
except Exception as e:
     print(f"Error running asyncio task: {e}")


Attempting to load HTML file from: file:///content/map.html
Successfully loaded /content/map.html
Screenshot saved to: /content/map.png
Browser and context closed.


In [41]:
# --- Example Parameters (USER: PLEASE DEFINE THESE) ---

# Define your Region of Interest (AOI) as a GeoJSON-like dictionary.
# Example: A small rectangle in an arbitrary location.
# REPLACE with your actual coordinates.
region_of_interest = poligon_AOI

# Define your date range for time-series data
example_start_date = '2000-01-01'
example_end_date = '2023-11-30' # For a shorter example run; extend as needed

# Define the year for land cover analysis
example_year_lc = '2020'

# Define the output directory for CSV files
output_csv_directory = 'gee_output_data'
# The functions will attempt to create this directory if it doesn't exist.

# Define the desired frequency for time-series aggregation
# Options: 'hourly', 'daily', 'monthly', 'yearly'
example_frequency = 'daily'

# --- Ensure output directory exists ---
os.makedirs(output_csv_directory, exist_ok=True) # Generic function handles this

# Extracting Albedo BSA
# print(f"--- Extracting Albedo BSA ({example_frequency}) ---")
bsa_path = extract_albedo_bsa(
    region_geojson=region_of_interest,
    start_date_str=example_start_date,
    end_date_str=example_end_date,
    frequency=example_frequency,
    output_dir=output_csv_directory
)

# Extracting Albedo WSA
print(f"--- Extracting Albedo WSA ({example_frequency}) ---")
wsa_path = extract_albedo_wsa(
    region_geojson=region_of_interest,
    start_date_str=example_start_date,
    end_date_str=example_end_date,
    frequency=example_frequency,
    output_dir=output_csv_directory
)

# print(f"--- Extracting Approximated Solar Radiation ({example_frequency}) ---")
# extract_radiacion_solar(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Day Temperature ({example_frequency}) ---")
# extract_temperatura_dia(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Night Temperature ({example_frequency}) ---")
# extract_temperatura_noche(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Wind Data ({example_frequency}) ---")
# # Note: Wind data from ERA5 is hourly. Generic function will average u/v components
# # to the target frequency before calculating speed/direction.
# extract_viento(
#     region_geojson=region_of_interest,
#     start_date_str=example_start_date,
#     end_date_str=example_end_date,
#     frequency=example_frequency, # e.g., 'daily' will average hourly components to daily means first
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Topography Data (Static) ---")
# extract_topography(
#     region_geojson=region_of_interest,
#     output_dir=output_csv_directory
# )

# print(f"--- Extracting Land Cover Data for {example_year_lc} ---")
# extract_land_cover(
#     region_geojson=region_of_interest,
#     year_str=example_year_lc,
#     output_dir=output_csv_directory
# )



No images found for Albedo_BSA in period 2000-01-01 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-02 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-03 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-04 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-05 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-06 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-07 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-08 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-09 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-10 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-11 for dataset MODIS/061/MCD43A3
No images found for Albedo_BSA in period 2000-01-12 for dataset MODIS/061/MCD43A3
No images found 

In [48]:
!pip install reportlab
!pip install html2image


In [65]:
import pandas as pd
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Table, TableStyle, Image
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.lib import colors
from reportlab.lib.units import inch
from datetime import datetime

# Convert HTML map to PNG and add to report
from html2image import Html2Image

# 1. Load and preprocess data
def load_data(bsa_path, wsa_path):
    bsa = pd.read_csv(bsa_path, parse_dates=['timestamp'])
    wsa = pd.read_csv(wsa_path, parse_dates=['timestamp'])
    df = pd.merge(bsa, wsa, on='timestamp', suffixes=('_BSA', '_WSA'))
    df['month'] = df['timestamp'].dt.month
    df['year'] = df['timestamp'].dt.year
    return df

# 2. Generate statistics tables
def calculate_statistics(df):
    monthly_stats = df.groupby('month')['Albedo_BSA'].agg([
        'mean', 'min', 'max',
        lambda x: x.quantile(0.01),  # p1
        lambda x: x.quantile(0.05),  # p5
        lambda x: x.quantile(0.10),  # p10
        lambda x: x.quantile(0.25),  # p25
        lambda x: x.quantile(0.50),  # median
        lambda x: x.quantile(0.75),  # p75
        lambda x: x.quantile(0.90),  # p90
        lambda x: x.quantile(0.95),  # p95
        lambda x: x.quantile(0.99),  # p99
    ])
    monthly_stats.columns = ['average', 'min', 'max', 'p1', 'p5', 'p10', 'p25',
                            'p50', 'p75', 'p90', 'p95', 'p99']
    return monthly_stats

# 3. Create visualizations
def create_figures(df, monthly_stats):
    # Figure 1: Monthly statistics
    plt.figure(figsize=(10, 6))
    monthly_stats[['average', 'min', 'max']].plot(kind='line', marker='o')
    plt.title('Monthly Albedo Statistics')
    plt.ylabel('Albedo')
    plt.xlabel('Month')
    plt.grid(True)
    plt.savefig('monthly_stats.png')
    plt.close()

    # Figure 2: Daily values timeline
    plt.figure(figsize=(12, 6))
    plt.plot(df['timestamp'], df['Albedo_BSA'], label='Daily Albedo')
    plt.title('Daily Albedo Values')
    plt.ylabel('Albedo')
    plt.xlabel('Date')
    plt.grid(True)
    plt.savefig('daily_values.png')
    plt.close()

# 4. Generate PDF report
def create_pdf_report(monthly_stats, annual_avg):
    doc = SimpleDocTemplate("Albedo_Report.pdf", pagesize=letter)
    styles = getSampleStyleSheet()
    elements = []

    # Title
    elements.append(Paragraph("Albedo Analysis Report", styles['Title']))
    elements.append(Spacer(1, 0.25*inch))

    # Add map image
    coordinates_text = str(poligon_AOI['coordinates'])
    elements.append(Paragraph("Study Area Location", styles['Heading2']))
    elements.append(Paragraph(coordinates_text, styles['BodyText']))
    elements.append(Image('map.png', width=6*inch, height=4*inch))
    elements.append(Spacer(1, 0.25*inch))

    # Methodology
    methodology = """
    <para align=justify>
    The final albedo was calculated using MODIS MCD43A3 data (Collection 6.1) including both
    black-sky albedo (BSA) and white-sky albedo (WSA) values. The analysis combines daily
    measurements from {start_year} to {end_year}, aggregated to monthly and annual statistics.
    The dataset shows albedo values ranging between {min_val:.2f} and {max_val:.2f} with an
    annual average of {avg_val:.2f}.
    </para>
    """.format(
        start_year=df['timestamp'].min().year,
        end_year=df['timestamp'].max().year,
        min_val=df['Albedo_BSA'].min(),
        max_val=df['Albedo_BSA'].max(),
        avg_val=annual_avg
    )
    elements.append(Paragraph(methodology, styles['BodyText']))
    elements.append(Spacer(1, 0.25*inch))

    # Monthly Statistics Table
    table_data = [['Month', 'Avg', 'Min', 'Max', 'P10', 'P50', 'P90']]
    for month, data in monthly_stats.iterrows():
        table_data.append([
            str(month),
            f"{data['average']:.2f}",
            f"{data['min']:.2f}",
            f"{data['max']:.2f}",
            f"{data['p10']:.2f}",
            f"{data['p50']:.2f}",
            f"{data['p90']:.2f}"
        ])

    t = Table(table_data)
    t.setStyle(TableStyle([
        ('BACKGROUND', (0,0), (-1,0), colors.grey),
        ('TEXTCOLOR', (0,0), (-1,0), colors.whitesmoke),
        ('ALIGN', (0,0), (-1,-1), 'CENTER'),
        ('FONTNAME', (0,0), (-1,0), 'Helvetica-Bold'),
        ('FONTSIZE', (0,0), (-1,0), 12),
        ('BOTTOMPADDING', (0,0), (-1,0), 12),
        ('BACKGROUND', (0,1), (-1,-1), colors.beige),
        ('GRID', (0,0), (-1,-1), 1, colors.black)
    ]))
    elements.append(t)
    elements.append(Spacer(1, 0.5*inch))

    # Add figures
    elements.append(Paragraph("Monthly Albedo Statistics", styles['Heading2']))
    elements.append(Image('monthly_stats.png', width=6*inch, height=4*inch))
    elements.append(Spacer(1, 0.25*inch))

    elements.append(Paragraph("Daily Albedo Values Timeline", styles['Heading2']))
    elements.append(Image('daily_values.png', width=6*inch, height=4*inch))

    doc.build(elements)

# Main workflow
if __name__ == "__main__":
    # Load your CSV files
    try:
      df = load_data(bsa_path, wsa_path)
    except:
      bsa_path = '/content/gee_output_data/Albedo_BSA_daily_20000101_20231130.csv'
      wsa_path = '/content/gee_output_data/Albedo_WSA_daily_20000101_20231130.csv'
      df = load_data(bsa_path, wsa_path)

    # Calculate statistics
    monthly_stats = calculate_statistics(df)
    annual_avg = df['Albedo_BSA'].mean()

    # Generate visualizations
    create_figures(df, monthly_stats)

    # Create PDF report
    create_pdf_report(monthly_stats, annual_avg)

<Figure size 1000x600 with 0 Axes>